## Aquisition de l'image

In [1]:
import subprocess

In [5]:
subprocess.run('fswebcam -r 1280x720 --no-banner images/bacATomates.jpg', shell=True)

--- Opening /dev/video0...
Trying source module v4l2...
/dev/video0 opened.
No input was specified, using the first.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to 'images/bacATomates.jpg'.


CompletedProcess(args='fswebcam -r 1280x720 --no-banner images/bacATomates.jpg', returncode=0)